In [1]:
import locale

#from matplotlib import pyplot as plt
import numpy as np

from db import psqlServer
sv = psqlServer()

control_chars = [chr(0x85)]
locale.setlocale(locale.LC_ALL, 'C')



'C'

In [2]:
def normalize_text(text):
    norm_text = text.lower()
    # Replace breaks with spaces
    norm_text = norm_text.replace('<br />', ' ')
    # Pad punctuation with spaces on both sides
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        norm_text = norm_text.replace(char, ' ' + char + ' ')
    return norm_text

In [3]:
AI = "arx_AI"        # artificial intelligence
FA = "arx_math_FA"   # functional analysis
GR = "arx_GRQC"      # general relativity and quantum cosmology
LG = "arx"           # learning
NT = "arx_math_NT"   # number theory

In [4]:
# How many items in all tables?
UNION = " UNION ALL "
SEL_C = "SELECT COUNT(*) as FOO FROM "
sum_statement = "SELECT SUM(FOO) FROM ("
sum_statement += SEL_C + AI
sum_statement += UNION + SEL_C + FA 
sum_statement += UNION + SEL_C + GR 
sum_statement += UNION + SEL_C + LG 
sum_statement += UNION + SEL_C + NT
sum_statement += ") as B;"

total = sv.execute(sum_statement)[0]['sum']
print(total)

116060


In [5]:
# Per table
SEL_C = "SELECT COUNT(*) FROM "
total_AI = sv.execute(SEL_C + AI)[0]['count']
total_FA = sv.execute(SEL_C + FA)[0]['count']
total_GR = sv.execute(SEL_C + GR)[0]['count']
total_LG = sv.execute(SEL_C + LG)[0]['count']
total_NT = sv.execute(SEL_C + NT)[0]['count']

print("AI:{}".format(total_AI))
print("FA:{}".format(total_FA))
print("GR:{}".format(total_GR))
print("LG:{}".format(total_LG))
print("NT:{}".format(total_NT))

AI:13000
FA:16000
GR:50000
LG:15060
NT:22000


In [6]:
# Get some rows from the AI table
rows = sv.execute("SELECT * FROM arx_AI")
corpus = [x['abstract'] for x in rows]
labels = [float(x['has_journal_ref']) for x in rows]

#rows2 = sv.execute("SELECT abstract FROM arx_math_FA LIMIT 10000")
#corpus += [x['abstract'] for x in rows2]
#labels += [x['has_journal_ref'] for x in rows2]

#rows3 = sv.execute("SELECT abstract FROM arx_GRQC LIMIT 30000")
#corpus += [x['abstract'] for x in rows3]
#labels += [x['has_journal_ref'] for x in rows2]


In [7]:
corpus_n = []
jj = 0
for corp in corpus:
    corpus_n.append(u"_*{0} {1}\n".format(jj, normalize_text(corp)))
    jj += 1

In [8]:
len(corpus_n)

13000

In [10]:
import gensim
from collections import namedtuple


In [11]:
SentimentDocument = namedtuple('SentimentDocument', 'words tags split sentiment')
alldocs = []
for line_no, line in enumerate(corpus_n[0:3250]):
    tokens = gensim.utils.to_unicode(corpus_n[line_no]).split()
    words = tokens[1:]
    tags = [line_no]
    split = 'train'
    #split = ['train', 'test', 'extra', 'extra'][line_no//12500]
    sentiment = labels[line_no]
    #sentiment = [1.0, 0.0, 1.0, 0.0, None, None, None, None][line_no//6750]
    alldocs.append(SentimentDocument(words, tags, split, sentiment))

for line_no, line in enumerate(corpus_n[3250:6500]):
    tokens = gensim.utils.to_unicode(corpus_n[line_no]).split()
    words = tokens[1:]
    tags = [line_no]
    split = 'test'
    #split = ['train', 'test', 'extra', 'extra'][line_no//12500]
    sentiment = labels[line_no]
    #sentiment = [1.0, 0.0, 1.0, 0.0, None, None, None, None][line_no//6750]
    alldocs.append(SentimentDocument(words, tags, split, sentiment))
    
for line_no, line in enumerate(corpus_n[6500:]):
    tokens = gensim.utils.to_unicode(corpus_n[line_no]).split()
    words = tokens[1:]
    tags = [line_no]
    split = 'extra'
    #split = ['train', 'test', 'extra', 'extra'][line_no//12500]
    sentiment = None
    #sentiment = [1.0, 0.0, 1.0, 0.0, None, None, None, None][line_no//6750]
    alldocs.append(SentimentDocument(words, tags, split, sentiment))    

In [12]:
len(alldocs)

13000

In [13]:
train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']
doc_list = alldocs[:]  # For reshuffling per pass

print('%d docs: %d train-sentiment, %d test-sentiment' % (len(doc_list), len(train_docs), len(test_docs)))

13000 docs: 3250 train-sentiment, 3250 test-sentiment


In [14]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

In [15]:
cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

In [16]:
simple_models = [
    # PV-DM w/ concatenation - window=5 (both sides) approximates paper's 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DBOW 
    Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DM w/ average
    Doc2Vec(dm=1, dm_mean=1, size=100, window=10, negative=5, hs=0, min_count=2, workers=cores),
]

In [17]:
simple_models[0].build_vocab(alldocs)

In [18]:
print(simple_models[0])
for model in simple_models[1:]:
    """Reuse shareable structures from other_model."""
    model.reset_from(simple_models[0])
    print(model)

Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)
Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)
Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)


In [19]:
models_by_name = OrderedDict((str(model), model) for model in simple_models)
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[2]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[0]])

In [20]:
models_by_name

OrderedDict([('Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)',
              <gensim.models.doc2vec.Doc2Vec at 0x10bbe7630>),
             ('Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)',
              <gensim.models.doc2vec.Doc2Vec at 0x10bbe76a0>),
             ('Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)',
              <gensim.models.doc2vec.Doc2Vec at 0x10bbe77f0>),
             ('dbow+dmm',
              <gensim.test.test_doc2vec.ConcatenatedDoc2Vec at 0x112e850b8>),
             ('dbow+dmc',
              <gensim.test.test_doc2vec.ConcatenatedDoc2Vec at 0x112e85080>)])

In [21]:

# Predictive Evaluation Methods

# Let's define some helper methods for evaluating the performance of our Doc2vec
# using paragraph vectors. We will classify document sentiments using a logistic 
# regression model based on our paragraph embeddings. We will compare the error 
# rates based on word embeddings from our various Doc2vec models.
import numpy as np
import statsmodels.api as sm
from random import sample

# For timing
from contextlib import contextmanager
from timeit import default_timer
import time 


/Users/alex/code/article-classifier/ac/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [22]:
@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    
def logistic_predictor_from_data(train_targets, train_regressors):
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    # print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set, infer=False, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets, train_regressors = zip(*[(doc.sentiment, test_model.docvecs[doc.tags[0]]) for doc in train_set])
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_data = test_set
    if infer:
        if infer_subsample < 1.0:
            test_data = sample(test_data, int(infer_subsample * len(test_data)))
        test_regressors = [test_model.infer_vector(doc.words, steps=infer_steps, alpha=infer_alpha) for doc in test_data]
    else:
        test_regressors = [test_model.docvecs[doc.tags[0]] for doc in test_docs]
    test_regressors = sm.add_constant(test_regressors)
    
    # Predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    
    # Here is the wtf: test_data is just [None]*500!!!
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_data])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

In [23]:

# Bulk Training

# We use an explicit multiple-pass, alpha-reduction approach as sketched in 
# this gensim doc2vec blog post with added shuffling of corpus on each pass.
# https://rare-technologies.com/doc2vec-tutorial/
from collections import defaultdict
best_error = defaultdict(lambda: 1.0)  # To selectively print only best errors achieved

In [24]:
from random import shuffle
import datetime

alpha, min_alpha, passes = (0.025, 0.001, 10)
alpha_delta = (alpha - min_alpha) / passes

print("START %s" % datetime.datetime.now())

for epoch in range(passes):
    shuffle(doc_list)  # Shuffling gets best results
    
    for name, train_model in models_by_name.items():
        # Train
        duration = 'na'
        train_model.alpha, train_model.min_alpha = alpha, alpha
        with elapsed_timer() as elapsed:
            train_model.train(doc_list, total_examples=len(doc_list), epochs=1)
            duration = '%.1f' % elapsed()
            
        # Evaluate
        eval_duration = ''
        with elapsed_timer() as eval_elapsed:
            err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs)
        eval_duration = '%.1f' % eval_elapsed()
        best_indicator = ' '
        if err <= best_error[name]:
            best_error[name] = err
            best_indicator = '*' 
        print("%s%f : %i passes : %s %ss %ss" % (best_indicator, err, epoch + 1, name, duration, eval_duration))

        if ((epoch + 1) % 5) == 0 or epoch == 0:
            eval_duration = ''
            with elapsed_timer() as eval_elapsed:
                infer_err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs, infer=True)
            eval_duration = '%.1f' % eval_elapsed()
            best_indicator = ' '
            if infer_err < best_error[name + '_inferred']:
                best_error[name + '_inferred'] = infer_err
                best_indicator = '*'
            print("%s%f : %i passes : %s %ss %ss" % (best_indicator, infer_err, epoch + 1, name + '_inferred', duration, eval_duration))

    print('Completed pass %i at alpha %f' % (epoch + 1, alpha))
    alpha -= alpha_delta
    
print("END %s" % str(datetime.datetime.now()))

START 2017-11-30 23:29:43.709040
*0.187077 : 1 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8) 3.2s 0.1s
*0.252308 : 1 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)_inferred 3.2s 0.7s
*0.188000 : 1 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t8) 1.2s 0.1s
*0.316923 : 1 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)_inferred 1.2s 0.3s
*0.190769 : 1 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8) 2.1s 0.1s
*0.384615 : 1 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)_inferred 2.1s 0.4s
*0.182462 : 1 passes : dbow+dmm 0.0s 0.3s
*0.381538 : 1 passes : dbow+dmm_inferred 0.0s 0.7s
*0.181846 : 1 passes : dbow+dmc 0.0s 0.3s
*0.347692 : 1 passes : dbow+dmc_inferred 0.0s 1.1s
Completed pass 1 at alpha 0.025000
*0.185846 : 2 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8) 3.2s 0.1s
*0.179385 : 2 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t8) 1.2s 0.1s
*0.188308 : 2 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8) 2.1s 0.1s
*0.177231 : 2 passes : dbow+dmm 0.0s 0.3s
*0.175385 : 2 passes : dbow+dmc 0.0

In [25]:
# Print best error rates achieved
print("Err rate Model")
for rate, name in sorted((rate, name) for name, rate in best_error.items()):
    print("%f %s" % (rate, name))


Err rate Model
0.168923 dbow+dmc
0.170462 dbow+dmm
0.177231 Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)
0.181538 Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)_inferred
0.182154 Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)
0.184923 Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)
0.187692 Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)_inferred
0.190769 dbow+dmc_inferred
0.196923 dbow+dmm_inferred
0.212308 Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)_inferred


In [26]:
doc_id = np.random.randint(simple_models[0].docvecs.count)  # Pick random doc; re-run cell for more examples
print('for doc %d...' % doc_id)
for model in simple_models:
    inferred_docvec = model.infer_vector(alldocs[doc_id].words)
    print('%s:\n %s' % (model, model.docvecs.most_similar([inferred_docvec], topn=3)))

for doc 762...
Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8):
 [(762, 0.4126853942871094), (1419, 0.4070093035697937), (3695, 0.40511882305145264)]
Doc2Vec(dbow,d100,n5,mc2,s0.001,t8):
 [(762, 0.9415596723556519), (6412, 0.659792959690094), (2939, 0.6571834683418274)]
Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8):
 [(762, 0.822860598564148), (3918, 0.8169604539871216), (5118, 0.8064792156219482)]


In [27]:

import random

doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(u'TARGET (%d): «%s»\n' % (doc_id, ' '.join(alldocs[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(alldocs[sims[index][0]].words)))



TARGET (1337): «this paper addresses the problem of minimizing a convex , lipschitz function $f$ over a convex , compact set $\xset$ under a stochastic bandit feedback model . in this model , the algorithm is allowed to observe noisy realizations of the function value $f ( x ) $ at any query point $x \in \xset$ . the quantity of interest is the regret of the algorithm , which is the sum of the function values at algorithm's query points minus the optimal function value . we demonstrate a generalization of the ellipsoid algorithm that incurs $\otil ( \poly ( d ) \sqrt{t} ) $ regret . since any algorithm has regret at least $\omega ( \sqrt{t} ) $ on this problem , our algorithm is optimal in terms of the scaling with $t$ .»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dbow,d100,n5,mc2,s0.001,t8):

MOST (3621, 0.7586348056793213): «ordinal regression is an important type of learning , which has properties of both classification and regression . here we describe a simple and effective approa

In [28]:
import random

doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(u'TARGET (%d): «%s»\n' % (doc_id, ' '.join(alldocs[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(alldocs[sims[index][0]].words)))



TARGET (4769): «in this paper , we describe our approach to the wikipedia participation challenge which aims to predict the number of edits a wikipedia editor will make in the next 5 months . the best submission from our team , " zeditor " , achieved 41 . 7% improvement over wmf's baseline predictive model and the final rank of 3rd place among 96 teams . an interesting characteristic of our approach is that only temporal dynamics features ( i . e . , how the number of edits changes in recent periods , etc . ) are used in a self-supervised learning framework , which makes it easy to be generalised to other application domains .»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8):

MOST (3647, 0.7816190719604492): «recent spectral clustering methods are a propular and powerful technique for data clustering . these methods need to solve the eigenproblem whose computational complexity is $o ( n^3 ) $ , where $n$ is the number of data samples . in this paper , a non-

In [29]:
import random

doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(u'TARGET (%d): «%s»\n' % (doc_id, ' '.join(alldocs[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(alldocs[sims[index][0]].words)))



TARGET (649): «this paper presents a new hybrid learning algorithm for unsupervised classification tasks . we combined fuzzy c-means learning algorithm and a supervised version of minimerror to develop a hybrid incremental strategy allowing unsupervised classifications . we applied this new approach to a real-world database in order to know if the information contained in unlabeled features of a geographic information system ( gis ) , allows to well classify it . finally , we compared our results to a classical supervised classification obtained by a multilayer perceptron .»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dbow,d100,n5,mc2,s0.001,t8):

MOST (4665, 0.7129030227661133): «this paper presents regression models obtained from a process of blind prediction of peptide binding affinity from provided descriptors for several distinct datasets as part of the 2006 comparative evaluation of prediction algorithms ( coepra ) contest . this paper finds that kernel partial least squares , a n